In [1]:
import matplotlib.pyplot as plt 
import rasterio 
import numpy as np 
import pandas as pd 
import os 
import glob

In [2]:
from upaths import prox_dpath,gedi_dpath,pred_dpath

In [3]:
tilename = "S02W064"

In [4]:
def get_tile_tifs(dpath, tilename):
    tile_files = glob.glob(f'{dpath}/{tilename}/*.tif')
    assert len(tile_files) > 0, 'No files found'
    return tile_files

def filter_12m_files(tile_files):
    tile_filex = [i for i in tile_files if '12.tif' in i]
    return tile_filex


def filter_files_byvar(tile_files, var):
    return [i for i in tile_files if var in i] 

In [6]:
gedi_files =  get_tile_tifs(gedi_dpath, tilename)
gedi_files = filter_12m_files(gedi_files)
gedi_dsme = filter_files_byvar(gedi_files, 'dsmE')[0]
gedi_dtme = filter_files_byvar(gedi_files, 'dtmE')[0]

gedi_dsmt = filter_files_byvar(gedi_files, 'dsmT')[0]
gedi_dtmt = filter_files_byvar(gedi_files, 'dtmT')[0]

#gedi_dsmt,gedi_dtmt,gedi_dtme,gedi_dsme

In [7]:
prox_files = get_tile_tifs(prox_dpath, tilename)
prox_edem = filter_files_byvar(prox_files, 'edem_W84.tif')[0]
prox_pdem = filter_files_byvar(prox_files, 'NegroAOIDTM.tif')[0]
prox_tdem = filter_files_byvar(prox_files, 'tdem_DEM.tif')[0]
prox_vdem = filter_files_byvar(prox_files, 'tdem_DEM__Fw.tif')[0]
prox_ldem = filter_files_byvar(prox_files, 'DTM_LiDAR.tif')[0]
#prox_pdem,prox_edem,prox_vdem,prox_tdem,prox_ldem 

In [9]:
pred_dname = "iter10000_n236435487_eqallxtile"
pred_tpath = f"{pred_dpath}/{pred_dname}"
pred_files = get_tile_tifs(pred_tpath, tilename)
pred_dtme = filter_files_byvar(pred_files, 'edem')[0]
#pred_dtme

In [63]:
# aggreae error min,max, rmse and stuff

In [64]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error, r2_score

def read_raster(file_path):
    """Reads a raster file and replaces nodata values with NaN."""
    with rasterio.open(file_path) as src:
        data = src.read(1)
        data = np.where(data == src.nodata, np.nan, data)
    return data

def compute_metrics(image1, image2):
    """Computes RMSE, R², SSIM, and PSNR between two images."""
    mask = ~np.isnan(image1) & ~np.isnan(image2)  # Ensure valid comparisons
    img1, img2 = image1[mask], image2[mask]

    rmse = np.sqrt(mean_squared_error(img1, img2))
    r2 = r2_score(img1, img2)
    ssim_value = ssim(img1.reshape(image1.shape), img2.reshape(image2.shape), data_range=img2.max() - img2.min())
    psnr_value = psnr(img1, img2, data_range=img2.max() - img2.min())

    return {"RMSE": rmse, "R²": r2, "SSIM": ssim_value, "PSNR": psnr_value}

def plot_rasters(dtm_path, dsm_path):
    """Reads, plots, and computes metrics for DTM and DSM rasters."""
    dtm = read_raster(dtm_path)
    dsm = read_raster(dsm_path)

    metrics = compute_metrics(dtm, dsm)
    print("Image Quality Metrics:", metrics)

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(dtm, cmap='terrain')
    ax[0].set_title("DTM (Bare Earth)")

    ax[1].imshow(dsm, cmap='terrain')
    ax[1].set_title("DSM (Surface Model)")

    plt.show()
    return metrics

# Example usage
# metrics = plot_rasters("path/to/tile_gdtm.tif", "path/to/tile_gdsm.tif")
